# Deep Dream : Inception ResNet V2
This jupyter notebook contains an implementation of DeepDream, a technique by Alexander Mordvintsev. <br>

Following things are covered:
<ol>
<li>Getting Data</li>
<li>Model Building</li>
</ol>

In [1]:
# import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import scipy.ndimage as nd
import warnings
import time
from PIL import Image
import shutil
import os
import cv2
from os.path import isfile, join
warnings.filterwarnings('ignore')

In [2]:
# use GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
# load model
inception_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet')

# summary
inception_model.summary()

219062272/219055592 [==============================] - 3s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_____________

In [4]:
# Maximize the activations of these layers
beg_names = ['mixed_5b','block35_1_mixed']          #Beginning of network
mid_names = ['mixed_6a','mixed_7a']                 #Mid of network
end_names = ['block8_9_mixed','block8_10_mixed']    #End of network

In [5]:
# Layers
layers = [inception_model.get_layer(name).output for name in beg_names]

# Create the feature extraction model
deepdream_inception_model = tf.keras.Model(inputs=inception_model.input, outputs=layers)

In [6]:
def calc_loss(image, model):
# Function used for loss calculations
# It works by feedforwarding the input image through the network and generate activations
# Then obtain the average and sum of those outputs

  img_batch = tf.expand_dims(image, axis=0) # Convert into batch format
  layer_activations = model(img_batch) # Run the model
  print('ACTIVATION VALUES (LAYER OUTPUT) =\n', layer_activations)
  # print('ACTIVATION SHAPE =\n', np.shape(layer_activations))

  losses = [] # accumulator to hold all the losses
  for act in layer_activations:
    loss = tf.math.reduce_mean(act) # calculate mean of each activation 
    losses.append(loss)
  
  print('LOSSES (FROM MULTIPLE ACTIVATION LAYERS) = ', losses)
  print('LOSSES SHAPE (FROM MULTIPLE ACTIVATION LAYERS) = ', np.shape(losses))
  print('SUM OF ALL LOSSES (FROM ALL SELECTED LAYERS)= ', tf.reduce_sum(losses))

  return  tf.reduce_sum(losses) # Calculate sum 

In [7]:
@tf.function
def deepdream(model, image, step_size):
    with tf.GradientTape() as tape:
      # This needs gradients relative to `img`
      # `GradientTape` only watches `tf.Variable`s by default
      tape.watch(image)
      loss = calc_loss(image, model) # call the function that calculate the loss 

    # Calculate the gradient of the loss with respect to the pixels of the input image.
    # The syntax is as follows: dy_dx = g.gradient(y, x) 
    gradients = tape.gradient(loss, image)

    print('GRADIENTS =\n', gradients)
    print('GRADIENTS SHAPE =\n', np.shape(gradients))

    # tf.math.reduce_std computes the standard deviation of elements across dimensions of a tensor
    gradients /= tf.math.reduce_std(gradients)  

    # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
    # You can update the image by directly adding the gradients (because they're the same shape!)
    
    image = image + gradients * step_size
    
    image = tf.clip_by_value(image, -1, 1)

    return loss, image

In [8]:
def deprocess(image):
  image = 255*(image + 1.0)/2.0
  return tf.cast(image, tf.uint8)

In [9]:
def run_deep_dream_simple(model, image, steps=100, step_size=0.01, dir = 0):
  # Convert from uint8 to the range expected by the model.
  h, w = image.shape[:2]
  s = 0.05 # scale coefficient
  image = tf.keras.applications.inception_v3.preprocess_input(image)
  image_shape = tf.constant(np.array(image))
  for step in range(steps):
    img = nd.affine_transform(image, [1-s,1-s,1], [h*s/2,w*s/2,0], order=2)
    loss, image = deepdream(model, img, step_size)
    
    if step % 1 == 0:
      url = r'./InceptionResNetV2/'+str(dir)+'/'+str(step)+'.jpg'
      im = Image.fromarray(np.array(deprocess(image)))
      im.save(url)
  return deprocess(image)

In [10]:
# shape of picture
np.shape(tf.keras.preprocessing.image.load_img(r'color.jpg'))

(1080, 1920, 3)

In [11]:
# remove folder
shutil.rmtree('./InceptionResNetV2')

# main folder
os.mkdir('./InceptionResNetV2')

In [12]:
# generate images
OCTAVE_SCALE = 1.1
Sample_Image = tf.keras.preprocessing.image.load_img(r'color.jpg', target_size = (1080, 1920))
img = tf.constant(np.array(Sample_Image))
base_shape = tf.shape(img)[:-1]
float_base_shape = tf.cast(base_shape, tf.float32)
i = 0
for n in range(0, 1):
  i = i + 1
  path = ""
  path = r"./InceptionResNetV2/"+ (str(i))
  print(path)
  os.mkdir(path)
  #print(n)
  new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32)
  img = tf.image.resize(img, new_shape).numpy()
  img = run_deep_dream_simple(model=deepdream_inception_model, image=img, steps=100, step_size=0.03, dir = str(i))

./InceptionV3_mid/1
ACTIVATION VALUES (LAYER OUTPUT) =
 [<tf.Tensor 'model/mixed_5b/concat:0' shape=(1, 132, 237, 320) dtype=float32>, <tf.Tensor 'model/block35_1_mixed/concat:0' shape=(1, 132, 237, 128) dtype=float32>]
LOSSES (FROM MULTIPLE ACTIVATION LAYERS) =  [<tf.Tensor 'Mean:0' shape=() dtype=float32>, <tf.Tensor 'Mean_1:0' shape=() dtype=float32>]
LOSSES SHAPE (FROM MULTIPLE ACTIVATION LAYERS) =  (2,)
SUM OF ALL LOSSES (FROM ALL SELECTED LAYERS)=  Tensor("Sum:0", shape=(), dtype=float32)
GRADIENTS =
 Tensor("gradient_tape/Reshape_3:0", shape=(1080, 1920, 3), dtype=float32)
GRADIENTS SHAPE =
 (1080, 1920, 3)


In [13]:
# generate movie

pathIn= 'InceptionResNetV2/1'
pathOut = 'InceptionV3_mid/1/InceptionResNetV2.avi'

frame_array = []
files = [f[:-4] for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = int)
for i in range(len(files)):
    #inserting the frames into an image array
    file_name = files[i] + '.jpg'
    frame_array.append(cv2.imread(os.path.join(pathIn, file_name)))

# fps
fps = 24

#reading each files
file_size = files[0] + '.jpg'
img = cv2.imread(os.path.join(pathIn, file_size))
height, width, layers = img.shape
size = (width,height)

out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [14]:
from google.colab import files
files.download('InceptionResNetV2/1/InceptionResNetV2.avi') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>